In [1]:
!pip install --quiet beautifulsoup4 html5lib

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

In [3]:
teams = [
    "ATL", "BOS", "BRK", "CHO", "CHI",
    "CLE", "DAL", "DEN", "DET", "GSW",
    "HOU", "IND", "LAC", "LAL", "MEM",
    "MIA", "MIL", "MIN", "NOP", "NYK",
    "OKC", "ORL", "PHI", "PHO", "POR",
    "SAC", "SAS", "TOR", "UTA", "WAS"
]

In [ ]:
all_dfs = []

for team in teams:
    # Roster
    url = f"https://www.basketball-reference.com/teams/{team}/2026.html#roster"
    roster_df = pd.read_html(url)[0]
    roster_df = roster_df.drop(columns=["No.", "College"])
    roster_df["Birth"] = roster_df["Birth"].apply(lambda x: x.split()[1])

    # Salaries
    salary_url = f"https://www.basketball-reference.com/contracts/{team}.html#contracts"
    salary_df = pd.read_html(salary_url)[0]
    salary_df = salary_df.droplevel(0, axis=1)
    salary_df = salary_df[["Player", "Age", "2025-26"]]
    salary_df = salary_df.dropna()
    salary_df = salary_df.copy()

    # Combine them
    combined_df = pd.merge(salary_df, roster_df, on="Player", how="outer").dropna()
    combined_df = combined_df.sort_values("Player")
    combined_df = combined_df.reset_index(drop=True)
    combined_df["PlayerLabel"] = combined_df["Player"].str.replace(" ", "\n", 1)
    combined_df["Salary"] = combined_df["2025-26"]
    combined_df = combined_df.drop(columns=["2025-26"])
    combined_df["Salary_number"] = (combined_df["Salary"].replace(r'[\$,]', '', regex=True).astype('float')) / 1000000
    combined_df["Team"] = team
    combined_df = combined_df.copy()
    
    all_dfs.append(combined_df)
    print(f"Added combined dataframe for Team: {team}, Length of list: {len(all_dfs)}")
    time.sleep(10)

Added combined dataframe for Team: ATL, Length of list: 1
Added combined dataframe for Team: BOS, Length of list: 2
Added combined dataframe for Team: BRK, Length of list: 3
Added combined dataframe for Team: CHO, Length of list: 4
Added combined dataframe for Team: CHI, Length of list: 5
Added combined dataframe for Team: CLE, Length of list: 6
Added combined dataframe for Team: DAL, Length of list: 7
Added combined dataframe for Team: DEN, Length of list: 8
Added combined dataframe for Team: DET, Length of list: 9
Added combined dataframe for Team: GSW, Length of list: 10
Added combined dataframe for Team: HOU, Length of list: 11
Added combined dataframe for Team: IND, Length of list: 12
Added combined dataframe for Team: LAC, Length of list: 13
Added combined dataframe for Team: LAL, Length of list: 14
Added combined dataframe for Team: MEM, Length of list: 15
Added combined dataframe for Team: MIA, Length of list: 16
Added combined dataframe for Team: MIL, Length of list: 17
Added 

In [5]:
len(all_dfs)

0

In [9]:
final_df = pd.concat(all_dfs)

In [ ]:
final_df.to_csv("final_csv")